In [ ]:
import keras
from keras.layers.core import Layer
import keras.backend as K
import tensorflow as tf
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten,BatchNormalization,Activation,MaxPooling2D

import cv2 
import numpy as np 

import pandas as pd

from keras.datasets import cifar10 
from keras import backend as K 
from keras.utils import np_utils
import os
import math 
from keras import optimizers
from keras.callbacks import LearningRateScheduler,ModelCheckpoint

In [ ]:
#  from google.colab import drive
#  drive.mount('/content/drive')
!unzip "/content/drive/My Drive/MRNet-v1.0" 

Archive:  /content/drive/My Drive/MRNet-v1.0.zip
warning [/content/drive/My Drive/MRNet-v1.0.zip]:  4294967296 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  4294967296
  (attempting to re-compensate)
replace MRNet-v1.0/valid-abnormal.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
def read_data(data_type, scan_type):
  list = []
  for filename in sorted(os.listdir("/content/MRNet-v1.0/"+data_type+"/"+scan_type)):
    if(filename != ".DS_Store"):
      instance = np.load("/content/MRNet-v1.0/"+data_type+"/"+scan_type+"/" + filename)
      mid = len(instance) // 2
      instance = np.array([ instance[mid-1] , instance[mid] ,instance[mid+1]])
      list.append(instance)
  data = np.asarray(list)
  data = np.moveaxis(data, 1, 3)
  print(scan_type+data_type, data.shape)
  return data

def read_labels(data_type, scan_type):
  path = "/content/MRNet-v1.0/"+data_type+"-"+scan_type + ".csv"
  df=pd.read_csv(path, sep=',',header=None)
  labels = df.values
  labels = labels[:,1]
  print(scan_type+data_type, labels.shape)
  return labels

print("Data: ")
sagittal_train_x = read_data("train","sagittal")
axial_train_x = read_data("train","axial")
coronal_train_x = read_data("train","coronal")

sagittal_test_x = read_data("valid","sagittal")
axial_test_x = read_data("valid","axial")
coronal_test_x = read_data("valid","coronal")

print("Labels: ")
abnormal_train_y = read_labels('train','abnormal')
acl_train_y = read_labels('train','acl')
meniscus_train_y = read_labels('train','meniscus')

abnornmal_test_y = read_labels('valid','abnormal')
acl_test_y = read_labels('valid','acl')
meniscus_test_y = read_labels('valid','meniscus')

stacked_labels_train = np.vstack((abnormal_train_y,acl_train_y))
stacked_labels_train = np.vstack((stacked_labels_train,meniscus_train_y)).T

stacked_labels_test = np.vstack((abnornmal_test_y,acl_test_y))
stacked_labels_test = np.vstack((stacked_labels_test,meniscus_test_y)).T

In [ ]:
def read_data_singles(data_type, scan_type, label_scan_type):
  list = []
  path = "/content/MRNet-v1.0/"+data_type+"-"+label_scan_type + ".csv"
  df=pd.read_csv(path, sep=',',header=None)
  labels = df.values
  labels = labels[:,1]
  single_labels = []
  i=0
  for filename in sorted(os.listdir("/content/MRNet-v1.0/"+data_type+"/"+scan_type)):
    if(filename != ".DS_Store"):
      instance = np.load("/content/MRNet-v1.0/"+data_type+"/"+scan_type+"/" + filename)
      for image in instance:
        list.append(np.array(image))
        single_labels.append(labels[i])
      i += 1
  data = np.asarray(list)
  single_labels = np.asarray(single_labels)
  print("single")
  print(scan_type+data_type, data.shape)
  print(label_scan_type+data_type, single_labels.shape)
  return data, single_labels

single_sagittal_train_x, single_saggital_abnormal_train_y = read_data_singles("train","sagittal","abnormal")
single_sagittal_test_x, single_saggital_abnormal_test_y = read_data_singles("valid","sagittal","abnormal")

In [ ]:

def conv2d_bn(x,filters,num_row,num_col, padding='same', strides=(1, 1), name=None):
   
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False
        )(x)

    x = BatchNormalization(axis = 3)(x)
    x = Activation('relu', name=name)(x)
    return x

In [ ]:
def inception_a(x,a1,a2,a3,b1,b2,b3,lname):
  b1x1_a1=conv2d_bn(x, a1, 1, 1)
  b3x3_a1_b1=conv2d_bn(b1x1_a1, b1, 3, 3)
  b3x3_a1_b2=conv2d_bn(b3x3_a1_b1, b1, 3, 3)

  b1x1_a2=conv2d_bn(x, a2, 1, 1)
  b3x3_a2_b1=b3x3_a1_b1=conv2d_bn(b1x1_a2, b2, 3, 3)

  bpool = AveragePooling2D((3, 3),strides=(1, 1),padding='same')(x)
  bpool_b1x1 = conv2d_bn(bpool, b3, 1, 1)

  b1x1_a3=conv2d_bn(x, a3, 1, 1)

  x = concatenate([b3x3_a1_b2, b3x3_a2_b1, bpool_b1x1, b1x1_a3], axis=3,name=lname)
  return x


def reduction_a(x):
  branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding='valid')
  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, strides=(2, 2), padding='valid')
  branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
  x = concatenate([branch3x3, branch3x3dbl, branch_pool],axis=3,name='reduction_a')
  return x


def inception_b(x,a1,b1,b12,a2,b2,b22,b3,a3,lname):

  b1x1_a1 = conv2d_bn(x, a1, 1, 1)
  b7x1_a1_b1 = conv2d_bn(b1x1_a1, b1, 7, 1)
  b1x7_a1_b1 = conv2d_bn(b7x1_a1_b1, b1, 1, 7)
  b7x1_a1_b2 = conv2d_bn(b1x7_a1_b1, b1, 7, 1)
  b1x7_a1_b2 = conv2d_bn(b7x1_a1_b2, b12, 1, 7)

  b1x1_a2 = conv2d_bn(x, a2, 1, 1)
  b1x7_a2_b1 = conv2d_bn(b1x1_a2, b2, 1, 7)
  b7x1_a2_b2 = conv2d_bn(b1x7_a2_b1, b22, 7, 1)

  bpool = AveragePooling2D((3, 3),strides=(1, 1),padding='same')(x)
  bpool_1x1 = conv2d_bn(bpool, b3, 1, 1)

  b1x1_a3 = conv2d_bn(x, a3, 1, 1)

  x = concatenate([b1x7_a1_b2, b7x1_a2_b2, bpool_1x1, b1x1_a3],axis=3,name=lname)

  return x

def reduction_b(x):
  branch3x3 = conv2d_bn(x, 192, 1, 1)
  branch3x3 = conv2d_bn(branch3x3, 320, 3, 3,strides=(2, 2), padding='valid')
  branch7x7x3 = conv2d_bn(x, 192, 1, 1)
  branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
  branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
  branch7x7x3 = conv2d_bn(branch7x7x3, 192, 3, 3, strides=(2, 2), padding='valid')
  branch_pool = MaxPool2D((3, 3), strides=(2, 2))(x)
  x = concatenate([branch3x3, branch7x7x3, branch_pool],axis=3,name='reduction_b')
  return x

def inception_c(x,i):
    branch1x1 = conv2d_bn(x, 320, 1, 1)
    branch3x3 = conv2d_bn(x, 384, 1, 1)
    branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
    branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
    branch3x3 = concatenate([branch3x3_1, branch3x3_2],axis=3,name='mixed9_' + str(i))

    branch3x3dbl = conv2d_bn(x, 448, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
    branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
    branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
    branch3x3dbl = concatenate([branch3x3dbl_1, branch3x3dbl_2], axis=3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = concatenate([branch1x1, branch3x3, branch3x3dbl, branch_pool],axis=3,name='inception_c' + str(i))
    return x

def aux_output(x):
  x1 = AveragePooling2D((5, 5), strides=3)(x)
  x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
  x1 = Flatten()(x1)
  x1 = Dense(1024, activation='relu')(x1)
  x1 = Dropout(0.7)(x1)
  x1 = Dense(1, activation='sigmoid', name='auxilliary_output_1')(x1)
  return x1

def stem(input_layer):
  x = conv2d_bn(input_layer, 32, 3, 3, strides=(2, 2), padding='valid')
  x = conv2d_bn(x, 32, 3, 3, padding='valid')
  x = conv2d_bn(x, 64, 3, 3)
  x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
  x = conv2d_bn(x, 80, 1, 1, padding='valid')
  x = conv2d_bn(x, 192, 3, 3, padding='valid')
  x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)
  return x

In [ ]:
def inception_v3(input_shape):
  input_layer = Input(shape=input_shape)

  x=stem(input_layer)

  x=inception_a(x,64,48,64,96,64,32,'inception_a1')

  x=inception_a(x,64,48,64,96,64,64,'inception_a2')

  x=inception_a(x,64,48,64,96,64,64,'inception_a3')

  x=reduction_a(x)

  x=inception_b(x,128,128,192,128,128,192,192,192,'inception_b1')

  for i in range(2):
    x=inception_b(x,160,160,192,160,160,192,192,192,('inception_b'+str(i+2)+""))

  x=inception_b(x,192,192,192,192,192,192,192,192,'inception_b5')


  x1=aux_output(x)

  x=reduction_b(x)


  for i in range(2):
    x=inception_c(x,i+1)

  x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)
  x = Dense(2048, activation='relu')(x)
  x = Dropout(0.4)(x)
  x = Dense(1, activation='sigmoid', name='output')(x)

  return input_layer,x,x1

In [ ]:
input_shape=(256, 256, 1)
inp,x,x1=inception_v3(input_shape)
Sagital_abnormal = Model(inp, [x, x1], name='inception_v3')


In [ ]:
Sagital_abnormal.summary()

In [ ]:
single_sagittal_train_x=single_sagittal_train_x.reshape((-1,256,256,1))
single_sagittal_train_x.shape

In [ ]:

sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
Sagital_abnormal.compile(loss=['binary_crossentropy', 'binary_crossentropy'], loss_weights=[1, 0.2], optimizer=sgd, metrics=['accuracy'])
checkpoint = ModelCheckpoint('drive/My Drive/sagital_acl.h5', monitor='val_output_accuracy', verbose=1, save_best_only=True, mode='max')
Sagital_abnormal.fit(
    x=single_sagittal_train_x,
    y=[single_saggital_abnormal_train_y,single_saggital_abnormal_train_y],
    epochs=15,
    verbose=1,
    validation_split=0.1,
    batch_size=32,
    callbacks=[checkpoint]  
    
)

In [ ]:


score = Sagital_abnormal.evaluate(sagittal_test_x, [abnornmal_test_y,abnornmal_test_y], verbose=0)
print( score)

[0.5117226541042328, 0.3861662745475769, 0.3705002963542938, 0.8166666626930237, 0.8416666388511658]


In [ ]:
print(model.metrics_names)

['loss', 'output_loss', 'auxilliary_output_1_loss', 'output_accuracy', 'auxilliary_output_1_accuracy']


In [ ]:
Sagital_abnormal.load_weights('drive/My Drive/sagital_acl.h5')


score = Sagital_abnormal.evaluate(sagittal_test_x, [abnornmal_test_y,abnornmal_test_y], verbose=0)
print( score)

[0.53804485698541, 0.39729127287864685, 0.386340469121933, 0.8166666626930237, 0.824999988079071]


In [ ]:
print(model.metrics_names)

['loss', 'output_loss', 'auxilliary_output_1_loss', 'output_accuracy', 'auxilliary_output_1_accuracy']
